In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import torch
import transformers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback

In [16]:
df = pd.read_csv("topic-analysis-dataset.csv")
print(df.head())

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['topic'])
num_labels = len(label_encoder.classes_)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'],
    test_size=0.2,
    random_state=42,
    stratify=df['label']  #to ensure equal class distribution
)

train_dataset = Dataset.from_dict({'text': train_texts.tolist(), 'label': train_labels.tolist()})
val_dataset = Dataset.from_dict({'text': val_texts.tolist(), 'label': val_labels.tolist()})

                                                text topic
0  usually , he would be tearing around the livin...  book
1  but just one look at a minion sent him practic...  book
2  that had been megan 's plan when she got him d...  book
3  he 'd seen the movie almost by mistake , consi...  book
4  she liked to think being surrounded by adults ...  book


In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

c:\Users\georg\anaconda3\envs\TextMining310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 15000/15000 [00:03<00:00, 3801.82 examples/s]


In [4]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
training_args = TrainingArguments(
    output_dir="./bert-topic-classifier",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # increase for speed on CPU with 32 GB RAM
    per_device_eval_batch_size=16,
    num_train_epochs=1,              # DistilBERT trains quickly, so 3 epochs is enough
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    fp16=True,  # disable if training on CPU
    disable_tqdm=False,
    seed=42
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

model.save_pretrained("./bert-topic-classifier")
tokenizer.save_pretrained("./bert-topic-classifier")

c:\Users\georg\anaconda3\envs\TextMining310\lib\site-packages\accelerate\accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
  1%|▏         | 51/3750 [00:06<07:24,  8.32it/s]

{'loss': 0.4743, 'grad_norm': 0.455658882856369, 'learning_rate': 1.9733333333333336e-05, 'epoch': 0.01}


  3%|▎         | 101/3750 [00:12<07:17,  8.33it/s]

{'loss': 0.0423, 'grad_norm': 0.12707911431789398, 'learning_rate': 1.9477333333333334e-05, 'epoch': 0.03}


  4%|▍         | 151/3750 [00:18<07:14,  8.28it/s]

...

{'loss': 0.0039, 'grad_norm': 0.0018155795987695456, 'learning_rate': 2.1333333333333336e-08, 'epoch': 1.0}


                                                   
100%|██████████| 3750/3750 [09:02<00:00,  8.23it/s]

{'eval_loss': 0.001749207847751677, 'eval_accuracy': 0.9994, 'eval_f1': 0.9993999799859982, 'eval_runtime': 34.7837, 'eval_samples_per_second': 431.236, 'eval_steps_per_second': 26.967, 'epoch': 1.0}


100%|██████████| 3750/3750 [09:03<00:00,  6.89it/s]


{'train_runtime': 543.9451, 'train_samples_per_second': 110.305, 'train_steps_per_second': 6.894, 'train_loss': 0.014095021969079972, 'epoch': 1.0}


In [20]:
modelpath = "bert-topic-classifier/checkpoint-3750" #cahnge if needed
model = AutoModelForSequenceClassification.from_pretrained(
    modelpath,
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

df = pd.read_csv("../test-datasets/sentiment-topic-test.tsv", sep="\t")
sentences = df["sentence"].tolist()
true_labels = df["topic"].tolist()
print(true_labels)
model.eval()
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

label_map = {0: "book", 1: "movie", 2: "sports"}
inverse_label_map = {v: k for k, v in label_map.items()}
predicted_labels = [label_map[p.item()] for p in predictions]
print(predicted_labels)
true_label_ids = [inverse_label_map[label] for label in true_labels]

print("Classification Report:")
print(classification_report(true_labels, predicted_labels))

print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels, labels=["negative", "neutral", "positive"]))

for index in range(len(sentences)):
    print(f"Sentence: {sentences[index]}, true label: {true_labels[index]}, predicted labels: {predicted_labels[index]}")


c:\Users\georg\anaconda3\envs\TextMining310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


['sports', 'sports', 'book', 'book', 'book', 'movie', 'movie', 'sports', 'sports', 'movie', 'book', 'book', 'movie', 'sports', 'sports', 'movie', 'movie', 'book']
['book', 'book', 'book', 'book', 'book', 'movie', 'book', 'book', 'book', 'book', 'book', 'book', 'movie', 'book', 'book', 'book', 'book', 'book']
Classification Report:
              precision    recall  f1-score   support

        book       0.38      1.00      0.55         6
       movie       1.00      0.33      0.50         6
      sports       0.00      0.00      0.00         6

    accuracy                           0.44        18
   macro avg       0.46      0.44      0.35        18
weighted avg       0.46      0.44      0.35        18


Confusion Matrix:


c:\Users\georg\anaconda3\envs\TextMining310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\georg\anaconda3\envs\TextMining310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\georg\anaconda3\envs\TextMining310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

ValueError: At least one label specified must be in y_true